In [1]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install line-bot-sdk flask flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.4 MB/s 
     |████████████████████████████████| 1.1 MB 39.0 MB/s 
     |████████████████████████████████| 144 kB 31.7 MB/s 
     |████████████████████████████████| 271 kB 40.2 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 


In [3]:
!pip install pyngrok
!pyngrok authtoken 29JlwqUGLZxPpC0QfilJEO8AwsW_xxCAFaGtukqQ8krP2CFd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=272c8d1fe7bfbbdd0521cb6a4cc2185cd409bcbd5b48ff7918a7f2d4275c1997
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
'''
引用套件
'''


# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [5]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("7EViQGhjLoXD6HnX+H2llWpD5+bO3sEXJZ/E85+X6YCwNud3ukbdZryinFuVzq0yoLlycjYLx4aqu/DwLhgiuCuzmWfN9Rqt93/4DbuaELQoORWV4OSAA/POyr7vkL+Ur96YcMEguV51FyKT5oVfoAdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("4c7bc83333634370cd348be01acb374d")

In [6]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [7]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          },
          {
            "type": "uri",
            "label": "url scheme",
            "uri": "https://line.me/R/nv/camera/"
          },
          {
            "type": "uri",
            "label": "打電話",
            "uri": "tel://3345678"
          }
        ],
  )
)
'''

'\nbuttons_template_message = TemplateSendMessage(\n    alt_text=\'Buttons template\',\n    template=ButtonsTemplate(\n        title=\'更多幫助\',\n        text=\'請點擊下方按鈕獲得更多幫助\',\n        actions=[\n          {\n            "type": "postback",\n            "label": "企業，查找商業結合方案",\n            "text": "[::text:]尋找BD",\n            "data": "Data1"\n          },\n          {\n            "type": "postback",\n            "label": "開發者，尋求教學",\n            "text": "[::text:]求助專家",\n            "data": "Data2"\n          },\n          {\n            "type": "uri",\n            "label": "url scheme",\n            "uri": "https://line.me/R/nv/camera/"\n          },\n          {\n            "type": "uri",\n            "label": "打電話",\n            "uri": "tel://3345678"\n          }\n        ],\n  )\n)\n'

In [8]:
# 測試1: 自訂按鍵範本訊息1 (LineDS照貼失敗)
'''
#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)
buttons_template_message = TemplateSendMessage(
  "type": "template",
  "altText": "this is a buttons template",
  "template": {
    "type": "buttons",
    "thumbnailImageUrl": "https://poarke.com/wp-content/uploads/24efe830375259.562088644ce1d.jpg",
    "imageBackgroundColor": "#CB4770",
    "title": "更多幫助",
    "text": "請點擊下方按鈕獲得更多幫助",
    "actions": [
      {
        "type": "message",
        "label": "企業，查找商業結合方案",
        "text": "[::text:]尋找BD"
      },
      {
        "type": "message",
        "label": "開發者，尋求教學",
        "text": "[::text:]求助專家"
      },
      {
        "type": "datetimepicker",
        "label": "選日期",
        "data": "我想選日期",
        "mode": "datetime",
        "initial": "2022-05-28T14:39",
        "max": "2023-05-28T14:39",
        "min": "2021-05-28T14:39"
      },
      {
        "type": "uri",
        "label": "url scheme",
        "uri": "https://line.me/R/nv/camera/"
      }
    ]
  }
)
'''

'\n#引入所需要的消息與模板消息\nfrom linebot.models import (\n    MessageEvent, TemplateSendMessage , PostbackEvent\n)\n\n#引入按鍵模板\nfrom linebot.models.template import(\n    ButtonsTemplate\n)\nbuttons_template_message = TemplateSendMessage(\n  "type": "template",\n  "altText": "this is a buttons template",\n  "template": {\n    "type": "buttons",\n    "thumbnailImageUrl": "https://poarke.com/wp-content/uploads/24efe830375259.562088644ce1d.jpg",\n    "imageBackgroundColor": "#CB4770",\n    "title": "更多幫助",\n    "text": "請點擊下方按鈕獲得更多幫助",\n    "actions": [\n      {\n        "type": "message",\n        "label": "企業，查找商業結合方案",\n        "text": "[::text:]尋找BD"\n      },\n      {\n        "type": "message",\n        "label": "開發者，尋求教學",\n        "text": "[::text:]求助專家"\n      },\n      {\n        "type": "datetimepicker",\n        "label": "選日期",\n        "data": "我想選日期",\n        "mode": "datetime",\n        "initial": "2022-05-28T14:39",\n        "max": "2023-05-28T14:39",\n        "min": "2021-05-28T14:

In [9]:
# 測試1: 自訂按鍵範本訊息2
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
# '''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        # type= "buttons",
        thumbnail_image_url='https://poarke.com/wp-content/uploads/24efe830375259.562088644ce1d.jpg',
        # imageBackgroundColor= "#CB4770",
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          },
          {
            "type": "uri",
            "label": "url scheme",
            "uri": "https://line.me/R/nv/camera/"
          },
          {
            "type": "uri",
            "label": "打電話",
            "uri": "tel://3345678"
          }
        ],
  )
)
# '''

In [10]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@more":buttons_template_message,
}

In [11]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@more")
        )

In [12]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    else:
        pass

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7321-35-231-255-2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/May/2022 12:55:07] "POST /callback HTTP/1.1" 404 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[]}


127.0.0.1 - - [30/May/2022 12:55:16] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16176288820155","text":"123"},"webhookEventId":"01G4AGJ30NYQK58QW819SKXW5T","deliveryContext":{"isRedelivery":false},"timestamp":1653915323122,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"71a91b4eb18c452091cab284b63f513b","mode":"active"}]}


127.0.0.1 - - [30/May/2022 12:55:24] "POST / HTTP/1.1" 200 -


{"destination":"Ufc94cda1b809d6632a36da9035938144","events":[{"type":"message","message":{"type":"text","id":"16176289346745","text":"@more"},"webhookEventId":"01G4AGJ8DJ9EMR6XRBJKAW27VA","deliveryContext":{"isRedelivery":false},"timestamp":1653915328898,"source":{"type":"user","userId":"Ua27054d19893e9f37f8babc1e76b6e2f"},"replyToken":"72d4c7fad79e4d20863f63f1b4329045","mode":"active"}]}


127.0.0.1 - - [30/May/2022 12:55:29] "POST / HTTP/1.1" 200 -
